In [21]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

학습 데이터를 만든다.

In [22]:
xData = [1,2,3,4,5,6,7]   # 근무시간
yData = [25000, 55000, 75000, 110000, 128000, 155000, 180000]   #근무 시간에 따른 매출 금액, 실제값

난수를 발생시켜 학습할 데이터의 기울기(가중치)와 y절편(바이어스)을 정한다.   
난수를 발생시켜서 작업하는 이유는 과적합을 방지하기 위해서이다.   
과적합이란 학습시킨 데이터에서는 높은 정확도를 보이지만 학습시킨 데이터 이회의 데이터에서는 낮은 정확도를 보이는 문제점을 말한다.    

In [23]:
# Variable(): tensorflow에서 사용할 변수를 선언한다.
# tf.random_uniform([난수의 갯수], 난수의 최소값, 난수의 최대값): tensorflow에서 사용할 난수를 발생시킨다.
a = tf.Variable(tf.random_uniform([1], -100, 100))  #가을기(과중치), -100 ~ 100 사이의 난수
b = tf.Variable(tf.random_uniform([1], -100, 100))  #y 절편 (바이어스), -100 ~ 100 사이의 난수
print('a = {}. b = {}'.format(a,b))

a = <tf.Variable 'Variable_4:0' shape=(1,) dtype=float32_ref>. b = <tf.Variable 'Variable_5:0' shape=(1,) dtype=float32_ref>


tensorflow에서 사용할 변수를 초기화하고 난수로 발생시킨 기울기와 y절편의 값을 확인한다.

In [24]:
# tensorflow 변수에 저장된 데이터를 출력하려면 tensorflow의 Session을 만들고 변수를 초기화 시킨 후 run() 함수를 이용한다.
# tensorflow 세션을 만든다 => 세션에서 실행한다
sess = tf.Session()
init = tf.global_variables_initializer()
# tensorflow 세션에서 run() 함수로 변수 초기화를 적용시킨다.
sess.run(init)
print('a = {}, b = {}'.format(sess.run(a),sess.run(b)))

a = [-20.727608], b = [-59.10115]


근무 시간과 매출 금액을 기억할 tensorflow에서 사용하는 자료형 (placeholder)을 선언한다.

In [25]:
x = tf.placeholder(dtype = tf.float32) # 근무시간(xData)를 기억하는 placeholder
y = tf.placeholder(dtype = tf.float32) # 매출금액(yData)를 기억하는 placeholder

1차 방정식 형태의 가설을 세우고 비용(오차) 함수를 정의한다.   

In [26]:
# 1차 방정식 형태의 가설,  Y => 예측값
Y = a * x + b
# 비용(오차) 함수 => 실제값(yData  => y라는 placeholder에 저장한다.)과 예측값(Y)의 편차의 제곱에 대한 평균을 이용한다.
# => 평균 제곱법
# reduce_mean(): tensorflow에서 평균을 계산하는 함수제곱값을 계산하는 함수
# square(): tensorflow에서 
cost = tf.reduce_mean(tf.square(Y - y)) # 예측값과 실제값의 편차에 제곱의 제곱에 대한 평균 = 오차

tensorflow의 경사 하강법 비용(오차) 함수의 결과를 가장 작게 만드는 방향으로 학습을 시키도록 정의한다.   

Gradient Descent is an optimization algorithm for finding a local minimum of a differentiable function. Gradient descent is simply used in machine learning to find the values of a function's parameters (coefficients) that minimize a cost function as far as possible.

In [27]:
# 경사 하강 알고리즘의 학습율을 설정한다
learning_rate = tf.Variable(0.01)
# train.GradientDescentOptimizer(학습율): tensorflow에서 학습율에서 학습율에 따른 비용(오차)를 계산하는 함수
optimizer = tf.train.GradientDescentOptimizer(learning_rate)   #경사하강을 paramater안에 있는 learning_rate 단위로 실행해라.
# minimize(): 경사하강 알고리즘에 따라 인수로 지정된 비용(오차) 함수의 최고값을 찾는다.
train = optimizer.minimize(cost)      #cost가 최소가 되는 값을 경사하강 learning_rate으로 찾아라.
# tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

학습시킨다 => 반복해서 수식을 실행한다.

In [33]:
# 세션을 만들고 변수를 초기화시킨다.
sees = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(5001):
    # 비용(오차) 함수를 가장 작게 만드는 학습을 할 수 있도록 placeholder에 데이터를 대입한다. 
    # 학습할 데이터는 반드시 'feed_dict'라는 이름의 딕셔너리 타입의 자료형을 만들어  대입해야 한다.
    # 'feed_dict' 딕셔너리의 key에는 placeholder의 이름을, value에는 각 placeholder에 대입할 리스트 타입의 데이터를 지정한다.
    # tensorflow 세션에서 비용(오차) 함수를 가장 작게 만드는 방향으로 'feed_dict' 딕셔너리의 데이터를 학습시킨다.
    sess.run(train, feed_dict= {x: xData, y: yData})
    
    # 500번 단위로 학습될 떄 마다 머신러닝이 실행되는 중간 결과를 확인한다.
    if i % 500 == 0:
        # index, 오차, 기울기, y절편
        print('index = {0:4d} ===> cost = {1:13.2f} ===> a (slope) = {2:10.2f} ===> b (bias) = {3:10.2f}'.format(i, sess.run(cost, feed_dict = {x: xData, y: yData}), sess.run(a)[0], sess.run(b)[0]))

index =    0 ===> cost = 4579015680.00 ===> a (slope) =   10377.88 ===> b (bias) =    2170.09
index =  500 ===> cost =    8690010.00 ===> a (slope) =   25534.73 ===> b (bias) =    1963.99
index = 1000 ===> cost =    8633873.00 ===> a (slope) =   25627.10 ===> b (bias) =    1506.59
index = 1500 ===> cost =    8632674.00 ===> a (slope) =   25640.56 ===> b (bias) =    1439.94
index = 2000 ===> cost =    8632654.00 ===> a (slope) =   25642.52 ===> b (bias) =    1430.23
index = 2500 ===> cost =    8632650.00 ===> a (slope) =   25642.80 ===> b (bias) =    1428.83
index = 3000 ===> cost =    8632658.00 ===> a (slope) =   25642.84 ===> b (bias) =    1428.63
index = 3500 ===> cost =    8632656.00 ===> a (slope) =   25642.85 ===> b (bias) =    1428.61
index = 4000 ===> cost =    8632656.00 ===> a (slope) =   25642.85 ===> b (bias) =    1428.61
index = 4500 ===> cost =    8632656.00 ===> a (slope) =   25642.85 ===> b (bias) =    1428.61
index = 5000 ===> cost =    8632656.00 ===> a (slope) =   25

3500    8632656.00   25642.85    1428.61    
4000    8632656.00   25642.85    1428.61    
4500    8632656.00   25642.85    1428.61    
5000    8632656.00   25642.85    1428.61    
기울기와 y 절판이 똑같다    
최적의 기울기 25642.85    최적의 y 절판 1428.6   
데이터 모델을 제일 정확하게 계산할 수 있는 a 값과 b 값이 계산됬다.

학습이 완료된 후 원하는 입력에 대한 결과를 출력한다

In [34]:
import time 
for i in range(8, 25): # 8시간부터 24시간 안쉬고 일하면 얼마를 벌 수 있을까?
    print('{0:2d}시간 근무했을 때 매출 기대금액: {1:7,.0f}'.format(i, sess.run(Y, feed_dict= {x: [i]})[0]))
    time.sleep(0.5)
# 학습후 제일 정확한 예측 모델 (a와 b를) 찾은후 더 많은 결과를 찾을 수 있다. 
# Y = a * x + b ==> a와 b는 학습결과로 찾았으면, 들어가야하는 데이터는 근무시간 x 말고는 없다.

 8시간 근무했을 때 매출 기대금액: 206,571
 9시간 근무했을 때 매출 기대금액: 232,214
10시간 근무했을 때 매출 기대금액: 257,857
11시간 근무했을 때 매출 기대금액: 283,500
12시간 근무했을 때 매출 기대금액: 309,143
13시간 근무했을 때 매출 기대금액: 334,786
14시간 근무했을 때 매출 기대금액: 360,428
15시간 근무했을 때 매출 기대금액: 386,071
16시간 근무했을 때 매출 기대금액: 411,714
17시간 근무했을 때 매출 기대금액: 437,357
18시간 근무했을 때 매출 기대금액: 463,000
19시간 근무했을 때 매출 기대금액: 488,643
20시간 근무했을 때 매출 기대금액: 514,286
21시간 근무했을 때 매출 기대금액: 539,928
22시간 근무했을 때 매출 기대금액: 565,571
23시간 근무했을 때 매출 기대금액: 591,214
24시간 근무했을 때 매출 기대금액: 616,857
